## Packages

In [ ]:
# Set the library path
#.libPaths("/user/emma.foessing01/u11969/new_R_libs")
Sys.setenv("PKG_CXXFLAGS"="-std=c++14")

print(R.version.string)

# List of required packages
list_of_packages <- c(
  "synthpop", "jsonlite", "codetools", "insight", "party", "haven", "dplyr", "rpart", "rpart.plot",
  "randomForest", "pROC", "caret", "pracma", "here", "Hmisc", "purrr",
  "ranger", "bnlearn", "arulesCBA", "network", "igraph", "xgboost",
  "data.table", "doParallel", "parallel", "ExtDist", "e1071"
)

# Function to load packages and handle errors
load_if_installed <- function(p) {
  tryCatch({
    library(p, character.only = TRUE)
  }, error = function(e) {
    message(sprintf("Package '%s' is not installed.", p))
  })
}

# Load all required packages
lapply(list_of_packages, load_if_installed)

## Data

In [ ]:
load(file = (paste0(here(), "/cpspop.RData")))
cpspop <- cpspop[, c(setdiff(names(cpspop), c("income", "race", "marital", "educ")), "income", "race", "marital", "educ")] #

adult <- read.csv(file = (paste0(here(),"/adult_preprocessed.csv")))
# delete NAs
adult[adult == "?"] <- NA
adult <- na.omit(adult)

adult$workclass <- as.factor(adult$workclass)
adult$education <- as.factor(adult$education)
adult$marital_status <- as.factor(adult$marital_status)
adult$relationship <- as.factor(adult$relationship)
adult$race <- as.factor(adult$race)
adult$sex <- as.factor(adult$sex)
adult$native_country <- as.factor(adult$native_country)
adult$income <- as.factor(adult$income)
adult$occupation <- as.factor(adult$occupation)

adult <- adult[, c("age", "fnlwgt", "capital_gain", "capital_loss", "hours_per_week", "income", "sex", "race", "relationship", "marital_status", "workclass", "occupation", "education", "native_country")]
adult[] <- lapply(adult, function(col) {
  if (is.integer(col)) {
    as.numeric(col)
  } else {
    col
  }
})

Ein Datensatz zu genieren (m = 1) ist ausreichend, da ich keine Varianzanalyse machen werde. Damit die Ergebnisse nicht von einem zufälligen Prozess abhängen ist es sinnvoll über ein paar runs Mittelwerte zu bilden (50–100)

## Functions

### Evaluation Functions

In [ ]:
## Calculate evaluation metrics for continuous targets
evaluation_metrics_cont <- function(predictions, test_set){
    # Residuals
    residuals <- predictions - test_set$income
    
    # Mean Absolute Error (MAE)
    MAE <- mean(abs(residuals))
    
    # Mean Squared Error (MSE) and Root Mean Squared Error (RMSE)
    MSE <- mean(residuals^2)
    RMSE <- sqrt(MSE)
    
    # R-squared: Guarding against zero variance in the target
    SS_res <- sum(residuals^2)
    SS_tot <- sum((test_set$income - mean(test_set$income))^2)
    R_squared <- ifelse(SS_tot == 0, NA, 1 - (SS_res / SS_tot))
    
    # Mean Absolute Percentage Error (MAPE): Handling division by zero
    MAPE <- ifelse(any(test_set$income == 0), NA, mean(abs(residuals / test_set$income)) * 100)
    
    metrics_df <- data.frame(
        MAE = MAE, 
        MSE = MSE, 
        RMSE = RMSE, 
        R_squared = R_squared, 
        MAPE = MAPE
    )
    
    return(metrics_df)
}

In [ ]:
## Calculate evaluation metrics for factored targets
evaluation_metrics_factor <- function(predictions, test_set) {
    # Ensure test_set is a data frame
    test_set <- as.data.frame(test_set)
    
    # Ensure both predictions and test_set$income are factors with the same levels
    predictions <- as.factor(predictions)
    reference <- as.factor(test_set$income)
    
    # Ensure levels match between predictions and reference
    levels(predictions) <- levels(reference)
    
    # Confusion matrix for the prediction on original data
    cm <- caret::confusionMatrix(predictions, reference, mode = "everything")

    # Saving evaluation metrics
    accuracy <- cm$overall['Accuracy']
    
    if (length(levels(reference)) == 2) {
        # Binary classification
        f1 <- cm$byClass['F1']
        sens <- cm$byClass['Sensitivity']
        spec <- cm$byClass['Specificity']
    } else {
        # Multi-class classification: calculate metrics for each class and take the mean
        f1 <- mean(cm$byClass[,'F1'], na.rm = TRUE)
        sens <- mean(cm$byClass[,'Sensitivity'], na.rm = TRUE)
        spec <- mean(cm$byClass[,'Specificity'], na.rm = TRUE)
    }

    # Create the dataframe
    metrics_df <- data.frame(
        Accuracy = accuracy, 
        F1 = f1, 
        Sensitivity = sens, 
        Specificity = spec
    )
    
    return(metrics_df)
}

## BN pred

In [ ]:
discretize_df <- function(df, breaks = 5) {
  for (var in colnames(df)) {
    if (!is.factor(df[[var]])) {
      freq_table <- table(df[[var]])
      zero_proportion <- ifelse(!is.na(freq_table[as.character(0)]), 
                                freq_table[as.character(0)] / sum(freq_table), 
                                0)

      new_breaks <- if (zero_proportion > 4/5) 1 else if (zero_proportion > 1/4) breaks - 2 else if (zero_proportion > 1/5) breaks - 1 else breaks
      
      zero_portion <- (df[[var]] == 0)
      non_zero_values <- as.numeric(as.character(df[[var]][!zero_portion]))

      if (length(non_zero_values) > 0) {
        range_values <- range(non_zero_values)
        breaks_values <- seq(range_values[1], range_values[2], length.out = new_breaks + 1)
        labels <- paste("(", head(breaks_values, -1), "-", tail(breaks_values, -1), "]", sep = "")

        discretized_non_zeros <- cut(non_zero_values, breaks = breaks_values, labels = labels, include.lowest = TRUE)
        df[[var]][!zero_portion] <- as.character(discretized_non_zeros)
      }
      df[[var]][zero_portion] <- "0"
      df[[var]] <- factor(df[[var]])
    }
  }
  return(df)
}


In [ ]:
inner_cv <- function(data, target_var, folds, algorithms) {
  # Create inner folds
  inner_folds <- createFolds(data[[target_var]], k = folds)
  
  best_model <- NULL
  best_performance <- -Inf
  best_algorithm <- NULL
  
  for (algorithm in algorithms) {
    cat("Trying algorithm:", algorithm, "\n")
    fold_results <- c()
    
    for (i in seq_along(inner_folds)) {
      inner_test_index <- inner_folds[[i]]
      inner_trainData <- data[-inner_test_index, ]
      inner_testData <- data[inner_test_index, ]
      
      # Fit Bayesian Network model using bnlearn algorithm
      bn_model <- do.call(get(algorithm, envir = asNamespace("bnlearn")), list(inner_trainData))
      
      # Fit the model to the training data
      fitted_bn_model <- bnlearn::bn.fit(bn_model, inner_trainData)
      
      # Use Bayesian Likelihood Weighting for prediction
      predictions <- predict(fitted_bn_model, node = target_var, data = inner_testData, method = "bayes-lw")
      
      # Handle missing levels in prediction
      predictions <- factor(predictions, levels = levels(inner_trainData[[target_var]]))
      
      # Calculate the accuracy
      accuracy <- mean(predictions == inner_testData[[target_var]], na.rm = TRUE)
      fold_results[i] <- accuracy
    }
    
    # Average performance for this algorithm
    avg_performance <- mean(fold_results, na.rm = TRUE)
    
    if (!is.na(avg_performance) && avg_performance > best_performance) {
      best_performance <- avg_performance
      best_model <- fitted_bn_model
      best_algorithm <- algorithm
    }
  }
  
  cat("Best algorithm selected:", best_algorithm, "with accuracy:", best_performance, "\n")
  return(best_model)
}

In [ ]:
bn_pred <- function(data, outer_folds, inner_folds) {
  # Discretize the data
  data <- discretize_df(data)
  
  algorithms <- c("tabu") # without hc
  data$income <- factor(data$income, levels = unique(data$income))
  
  outer_cv_folds <- createFolds(data$income, k = outer_folds)
  
  # Setup parallel backend
  num_cores <- detectCores() - 1  # You can adjust this depending on available cores
  cl <- makeCluster(num_cores)
  registerDoParallel(cl)
  
  # Use foreach for parallelization of outer folds
  outer_results <- foreach(i = seq_along(outer_cv_folds), 
                           .packages = c("caret", "dplyr", "bnlearn", "doParallel", "foreach", "parallel"), 
                           .export = c("discretize_df", "inner_cv", "evaluation_metrics_factor")) %dopar% {
    outer_test_index <- outer_cv_folds[[i]]
    outer_testData <- data[outer_test_index, ]
    outer_trainData <- data[-outer_test_index, ]
    
    # Get the best fitted model from inner CV
    best_model <- inner_cv(outer_trainData, "income", inner_folds, algorithms)
    
    # Perform prediction using 'bayes-lw' method
    predictions <- predict(best_model, node = "income", data = outer_testData, method = "bayes-lw")
    
    # Ensure both predictions and test data are factors
    predictions <- factor(predictions, levels = levels(outer_testData$income))
    
    cat("Class of outer_testData: ", class(outer_testData), "\n")

    # Evaluate predictions (ensure test data is also a factor)
    eval <- evaluation_metrics_factor(predictions, outer_testData)
    
    return(eval)
  }
  
  stopCluster(cl)  # Stop the cluster when done
  registerDoSEQ()

  # Average the evaluation metrics over the outer folds
  eval_avg_outer_folds <- do.call(rbind, outer_results) %>%
                          dplyr::summarise(across(everything(), mean, na.rm = TRUE))
  
  return(eval_avg_outer_folds)
}


In [ ]:
s <- 1236
set.seed(s) 

In [ ]:
syndata <- readRDS(paste0(here(), "/results/", "cps", "_svm_", as.character(s), ".rds"))
BN_eval <- bn_pred(syndata, outer_folds = 5, inner_folds = 3)

In [ ]:
BN_eval